In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

import sys,os
sys.path.append('../')
from collections import Counter

from copy import copy,deepcopy

import numpy as np
import pandas as pd


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,RobustScaler
from sklearn import preprocessing

from efcm.cluster import EFCM_LS1

### Load data

In [ ]:
dataset_names = ['dataset_01_scaled', 'dataset_02_scaled', 'dataset_03_scaled']
data_01 = pd.read_csv(os.path.join('../input/', '.'.join([dataset_names[0], 'csv'])))
data_02 = pd.read_csv(os.path.join('../input/', '.'.join([dataset_names[1], 'csv'])))
data_03 = pd.read_csv(os.path.join('../input/', '.'.join([dataset_names[2], 'csv'])))

### View data

In [ ]:
data_01.head()

In [ ]:
data_02.head()

In [ ]:
data_03.head()

In [ ]:
print('Shape',data_01.shape,data_02.shape,data_03.shape)

In [ ]:
X_data_01 = data_01.drop(columns='Target').values.copy()
y_data_01 = data_01.Target.values.copy()
X_data_02 = data_02.drop(columns='Target').values.copy()
y_data_02 = data_02.Target.values.copy()
X_data_03 = data_03.drop(columns='Target').values.copy()
y_data_03 = data_03.Target.values.copy()

In [ ]:
try:
    assert y_data_01.all() == y_data_02.all() ==y_data_03.all()
    y = y_data_01
except:
    print('Target invalid')

### Parâmetros de T_u e T_V no diretório -- ../run_best_params

 - Dataset_01 : T_u=0.43  T_v=10.0
 - Dataset_02 : T_u=2.55  T_v=10.0
 - Dataset_03 : T_u=0.05  T_v=10.0

### Evaluate

In [ ]:
clf = EFCM_LS1(n_cluster=7,theta_u=.43 ,theta_v=10,epsilon=1e-10,tol_iter=150,init='laplace')
u = []
epoch = 1 # mudar para 50
model = []
_ = [model.append(deepcopy(clf)) for _ in range(epoch)]
for j in range(epoch):
    model[j].fit(X_data_01) # X_data_01,X_data_02,X_data_03
    print('run:{} loss: {} '.format(j,model[j].loss))
    u.append(model[j].loss)

In [ ]:
u = np.array(u)
np.argmin(u),u[np.argmin(u)]

### Partition Fuzzy Matrix

In [ ]:
model[np.argmin(u)].fuzzy_matrix

### Matrix V

In [ ]:
model[np.argmin(u)].fuzzy_weight_matrix

### Labels

In [ ]:
model[np.argmin(u)].labels_

In [ ]:
Counter(model[np.argmin(u)].labels_)

### Loss History

In [ ]:
# plot
plt.plot(model[np.argmin(u)].loss_history)